In [1]:
import bentoml
import csv
from datetime import datetime
import os

import matplotlib.pyplot as plt
import mlflow
import mlflow.tensorflow
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import minmax_scale
from source.custom_layer import NBeatsBlock
from source.utils import *
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import (
    Dense,
    Dropout,
    Lambda,
    Conv1D,
    Input,
    LSTM,
    subtract,
    add
    )
from tensorflow.keras.utils import plot_model

2023-10-30 08:22:32.857134: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-30 08:22:35.016580: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-30 08:22:35.029295: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-30 08:22:37.880860: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:

X_train, X_test, y_train, y_test = load_data(save_dir='../data/features')

In [3]:
bitcoin_prices, timesteps, prices = load_btc_data()

In [4]:
# predict 1 step at a time
HORIZON = 1
# use a week worth of timesteps to predict the horizon
WINDOW_SIZE = 7

In [5]:
full_windows, full_labels = make_windows(
    prices,
    window_size=WINDOW_SIZE,
    horizon=HORIZON
)

In [6]:
train_windows, test_windows, train_labels, test_labels = make_train_test_splits(
    full_windows,
    full_labels)

In [11]:
tf.random.set_seed(42)

model_1 = tf.keras.Sequential(
    [
        
        Dense(128, activation='relu', input_shape=(WINDOW_SIZE,)),
        Dense(HORIZON, activation='linear')
    ],
    name='model_1_dense'
)

model_1.compile(
    loss = 'mae',
    optimizer=Adam(),
    metrics=['mae']
)

In [12]:
model_1.build()

In [13]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 128)               1024      
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1153 (4.50 KB)
Trainable params: 1153 (4.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
model_1.fit(
        x=train_windows,
        y=train_labels,
        epochs=100,
        batch_size=128,
        verbose=0,
        validation_data=(test_windows, test_labels)
    )

In [16]:
test_windows[:1]

array([[9290.89660239, 9202.41545055, 9369.62808116, 9326.59962378,
        9335.75240233, 9226.48582088, 8794.35864452]])

In [15]:
bentoml.tensorflow.save_model(
    'bit-predict-final',
    model_1,
    signatures={"__call__": {"batchable": True, "batch_dim": 0}}
)

INFO:tensorflow:Assets written to: /tmp/tmppuq70opdbentoml_model_bit-predict-final/assets


INFO:tensorflow:Assets written to: /tmp/tmppuq70opdbentoml_model_bit-predict-final/assets


Model(tag="bit-predict-final:jodqzbdxc644sz7a", path="/home/pop/bentoml/models/bit-predict-final/jodqzbdxc644sz7a/")